In [ ]:
# Visualize the provided 3D trajectory and speed (static 3D plot + speed-over-time + optional GIF animation)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.animation import FuncAnimation, PillowWriter

# ----- User-supplied parameters/code -----
MAX_SPEED     = 12.0   # m/s (hard cap)
MAX_ACCEL     = 3.0    # m/s^2 (hard cap)
MARGIN        = 0.995  # safety factor vs theory bounds
T             = 130.0  # duration (s)
DT            = 0.02   # timestep (s)
Z0            = 10.0   # altitude offset (m)
SEED          = 183    # RNG seed (change for a different “random” path)

# Component counts per axis (more = richer motion)
NX, NY, NZ = 8, 8, 4

# Random ranges (feel free to tweak)
# Frequencies in Hz (periods ~50–1000 s); vertical a tad slower
FX_RANGE = (0.0002, 0.003)
FY_RANGE = (0.0001, 0.0017)
FZ_RANGE = (0.0015, 0.010)

# Amplitude ranges in meters (horizontal > vertical)
AX_RANGE = (15.0, 60.0)
AY_RANGE = (12.0, 50.0)
AZ_RANGE = (4.0,  18.0)

def rand_components(n, frange, arange, rng):
    f  = rng.uniform(*frange, size=n)     # Hz
    A  = rng.uniform(*arange, size=n)     # m
    ph = rng.uniform(0, 2*np.pi, size=n)  # rad
    return A, f, ph

def build_soft_trajectory(t):
    rng = np.random.default_rng(SEED)

    Ax, fx, phx = rand_components(NX, FX_RANGE, AX_RANGE, rng)
    Ay, fy, phy = rand_components(NY, FY_RANGE, AY_RANGE, rng)
    Az, fz, phz = rand_components(NZ, FZ_RANGE, AZ_RANGE, rng)

    wx, wy, wz = 2*np.pi*fx, 2*np.pi*fy, 2*np.pi*fz

    # Worst-case bounds (independent of phases)
    Dx  = np.sum(Ax*wx);   Dy  = np.sum(Ay*wy);   Dz  = np.sum(Az*wz)
    Bv  = np.sqrt(Dx**2 + Dy**2 + Dz**2)

    D2x = np.sum(Ax*wx**2); D2y = np.sum(Ay*wy**2); D2z = np.sum(Az*wz**2)
    Ba  = np.sqrt(D2x**2 + D2y**2 + D2z**2)

    S_speed = (MARGIN*MAX_SPEED) / max(Bv, 1e-12)
    S_accel = (MARGIN*MAX_ACCEL) / max(Ba, 1e-12)
    S0 = min(S_speed, S_accel)

    def mix_sin(a, w, ph):    return (a[None,:]*np.sin(w[None,:]*t[:,None] + ph[None,:])).sum(axis=1)
    def mix_cos_w(a, w, ph):  return (a[None,:]*w[None,:]*np.cos(w[None,:]*t[:,None] + ph[None,:])).sum(axis=1)
    def mix_sin_w2(a, w, ph): return (a[None,:]*w[None,:]**2*np.sin(w[None,:]*t[:,None] + ph[None,:])).sum(axis=1)

    # Position / velocity / acceleration (analytic)
    x  = S0*mix_sin(Ax, wx, phx);   y  = S0*mix_sin(Ay, wy, phy);   z  = Z0 + S0*mix_sin(Az, wz, phz)
    vx = S0*mix_cos_w(Ax, wx, phx); vy = S0*mix_cos_w(Ay, wy, phy); vz = S0*mix_cos_w(Az, wz, phz)
    ax = -S0*mix_sin_w2(Ax, wx, phx); ay = -S0*mix_sin_w2(Ay, wy, phy); az = -S0*mix_sin_w2(Az, wz, phz)

    vmag = np.sqrt(vx**2 + vy**2 + vz**2)
    amag = np.sqrt(ax**2 + ay**2 + az**2)

    # Gentle nudge toward (but below) the caps
    target_v = 0.98*MAX_SPEED
    target_a = 0.90*MAX_ACCEL
    k_speed_theory = (MARGIN*MAX_SPEED)/(S0*Bv) if Bv>0 else 1.0
    k_accel_theory = (MARGIN*MAX_ACCEL)/(S0*Ba) if Ba>0 else 1.0
    k_speed_sample = (target_v/vmag.max()) if vmag.max()>0 else 1.0
    k_accel_sample = (target_a/amag.max()) if amag.max()>0 else 1.0
    k = min(k_speed_theory, k_accel_theory, k_speed_sample, k_accel_sample)

    if k != 1.0:
        x, y, z  = k*x, k*y, Z0 + k*(z - Z0)
        vx,vy,vz = k*vx, k*vy, k*vz
        ax,ay,az = k*ax, k*ay, k*az
        vmag     = np.sqrt(vx**2 + vy**2 + vz**2)
        amag     = np.sqrt(ax**2 + ay**2 + az**2)

    v_bound = (S0*k)*Bv
    a_bound = (S0*k)*Ba
    return x, y, z, vx, vy, vz, v_bound, a_bound


In [ ]:

# ----- Generate data -----
t = np.arange(0.0, T, DT)
x, y, z, vx, vy, vz, v_bound, a_bound = build_soft_trajectory(t)
speed = np.sqrt(vx**2 + vy**2 + vz**2)

# ----- 3D curve colored by speed -----
fig = plt.figure(figsize=(8, 6))
ax3d = fig.add_subplot(111, projection='3d')

# Build line segments for a multicolored 3D line
points = np.array([x, y, z]).T.reshape(-1, 1, 3)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = Normalize(vmin=speed.min(), vmax=speed.max())
lc = Line3DCollection(segments, cmap=cm.viridis, norm=norm)
lc.set_array(speed[:-1])
lc.set_linewidth(2.0)
ax3d.add_collection3d(lc)

# Add start/end markers
ax3d.scatter([x[0]], [y[0]], [z[0]], s=30)   # start
ax3d.scatter([x[-1]],[y[-1]],[z[-1]], s=30)  # end

# Set axes limits for a nice view
ax3d.set_xlim(np.min(x), np.max(x))
ax3d.set_ylim(np.min(y), np.max(y))
ax3d.set_zlim(np.min(z), np.max(z))
ax3d.set_xlabel("X [m]")
ax3d.set_ylabel("Y [m]")
ax3d.set_zlabel("Z [m]")
cb = fig.colorbar(lc, ax=ax3d, fraction=0.03, pad=0.1)
cb.set_label("Speed [m/s]")
plt.title("3D Trajectory colored by speed")
plt.show()

# ----- Speed vs time plot -----
plt.figure(figsize=(8, 4.5))
plt.plot(t, speed, linewidth=1.5)
plt.axhline(MAX_SPEED, linestyle='--')
plt.xlabel("Time [s]")
plt.ylabel("Speed [m/s]")
plt.title("Speed over time")
plt.show()

# ----- Optional: short GIF animation (moving point + trail) -----
# Keep frames moderate so it renders quickly
sample_step = 4  # use every Nth frame to reduce GIF size
idx = np.arange(0, len(t), sample_step)

fig_anim = plt.figure(figsize=(6, 5))
ax_anim = fig_anim.add_subplot(111, projection='3d')
ax_anim.set_xlim(np.min(x), np.max(x))
ax_anim.set_ylim(np.min(y), np.max(y))
ax_anim.set_zlim(np.min(z), np.max(z))
ax_anim.set_xlabel("X [m]"); ax_anim.set_ylabel("Y [m]"); ax_anim.set_zlabel("Z [m]")
ax_anim.set_title("Trajectory animation (point + trail)")

trail_len = 60  # how many points to keep as a trailing tail (in sampled frames)
point_plot = ax_anim.plot([x[0]], [y[0]], [z[0]], marker='o')[0]
trail_plot, = ax_anim.plot([], [], [], linewidth=2.0)




In [ ]:
# ----- Generate data -----
t = np.arange(0.0, T, DT)
x, y, z, vx, vy, vz, v_bound, a_bound = build_soft_trajectory(t)
speed = np.sqrt(vx**2 + vy**2 + vz**2)

# ----- 3D trajectory colored by speed (no Line3DCollection) -----
fig = plt.figure(figsize=(8, 6))
ax3d = fig.add_subplot(111, projection='3d')

# optional faint path for continuity
ax3d.plot(x, y, z, linewidth=0.8, alpha=0.4)

# color each point by its instantaneous speed
sc = ax3d.scatter(x, y, z, c=speed, cmap='viridis', s=6)
ax3d.scatter([x[0]],[y[0]],[z[0]], s=40)   # start
ax3d.scatter([x[-1]],[y[-1]],[z[-1]], s=40)  # end

ax3d.set_xlim(np.min(x), np.max(x))
ax3d.set_ylim(np.min(y), np.max(y))
ax3d.set_zlim(np.min(z), np.max(z))
ax3d.set_xlabel("X [m]"); ax3d.set_ylabel("Y [m]"); ax3d.set_zlabel("Z [m]")
cb = fig.colorbar(sc, ax=ax3d, fraction=0.03, pad=0.1)
cb.set_label("Speed [m/s]")
plt.title("3D Trajectory colored by speed")
plt.show()

# ----- Speed vs time -----
plt.figure(figsize=(8, 4.5))
plt.plot(t, speed, linewidth=1.5)
plt.axhline(MAX_SPEED, linestyle='--')
plt.xlabel("Time [s]"); plt.ylabel("Speed [m/s]")
plt.title("Speed over time")
plt.show()

# ----- Animation (point + trail), color point by speed -----
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import cm

sample_step = 4
idx = np.arange(0, len(t), sample_step)

fig_anim = plt.figure(figsize=(6, 5))
ax_anim = fig_anim.add_subplot(111, projection='3d')
ax_anim.set_xlim(np.min(x), np.max(x))
ax_anim.set_ylim(np.min(y), np.max(y))
ax_anim.set_zlim(np.min(z), np.max(z))
ax_anim.set_xlabel("X [m]"); ax_anim.set_ylabel("Y [m]"); ax_anim.set_zlabel("Z [m]")
ax_anim.set_title("Trajectory animation (point + trail)")

trail_len = 60
point_plot, = ax_anim.plot([x[0]],[y[0]],[z[0]], marker='o', linestyle='None')
trail_plot, = ax_anim.plot([], [], [], linewidth=2.0)


# Save if you want a GIF
# anim.save("trajectory.gif", writer=PillowWriter(fps=30))


In [ ]:
csv_text = """sample_stamp_sec,sample_stamp_nanosec,pose1_stamp_sec,pose1_stamp_nanosec,pose1_x,pose1_y,pose1_z,pose2_stamp_sec,pose2_stamp_nanosec,pose2_x,pose2_y,pose2_z,distance,distance_sum
1755376960,428161922,1755376960,390381184,24.25164222717285,11.28261661529541,60.7172966003418,1755376960,383330882,28.21200090566134,12.343229489839743,75.75344237475124,15.585090,15.585090
1755376960,928159529,1755376960,890636416,24.885475158691406,11.454756736755371,62.63907241821289,1755376960,882940953,29.03146129937167,12.580348193333307,78.2695513350939,16.210121,31.795211
1755376961,428167044,1755376961,390888832,25.52912712097168,11.629143714904785,64.59159088134766,1755376961,382899563,29.850957260731832,12.818190196975152,80.77044283739333,16.788308,48.583519
1755376961,928176089,1755376961,893147392,26.187437057495117,11.807598114013672,66.59850311279297,1755376961,882916173,30.669934096048507,13.056623714827047,83.25306984240649,17.292410,65.875929
1755376962,428184977,1755376962,391270784,26.845869064331055,11.987785339355469,68.61933898925781,1755376962,382896263,31.488136111528036,13.295602713224412,85.71533798329031,17.763283,83.639212
1755376962,928175534,1755376962,891523712,27.511749267578125,12.173385620117188,70.67359924316406,1755376962,883017255,32.30578752967986,13.53522133891488,88.15661595520739,18.179474,101.818686
1755376963,428167033,1755376963,391648640,28.182233810424805,12.362825393676758,72.74693298339844,1755376963,382892337,33.122130679459076,13.775285696838083,90.57336864796612,18.552074,120.370759
1755376963,928174279,1755376963,893774592,28.86427879333496,12.559197425842285,74.85855865478516,1755376963,882964485,33.937841051630215,14.016022749175292,92.96634035147747,18.861472,139.232231
1755376964,428173369,1755376964,392030336,29.546005249023438,12.756436347961426,76.96966552734375,1755376964,383001977,34.75242563917175,14.257315904380086,95.33283733524542,19.145901,158.378133
1755376964,928172046,1755376964,892281216,30.233901977539062,12.956122398376465,79.10630798339844,1755376964,882940101,35.56570299657352,14.49913954067707,97.67112946012168,19.376832,177.754964
1755376965,428217529,1755376965,392408448,30.929906845092773,13.157955169677734,81.24761199951172,1755376965,382919675,36.377814793974665,14.741564151376423,99.98044004960397,19.573103,197.328067
1755376965,928168725,1755376965,894528768,31.641986846923828,13.366575241088867,83.41258239746094,1755376965,882921443,37.18864144448443,14.984582777357488,102.25926144952824,19.712449,217.040516
1755376966,428199646,1755376966,392776064,32.35672378540039,13.576752662658691,85.55130004882812,1755376966,382919133,37.998053686941006,15.228185317960062,104.50609284209025,19.845302,236.885818
1755376966,928187536,1755376966,893028736,33.082767486572266,13.789216995239258,87.7034912109375,1755376966,882922392,38.80597892285806,15.472378545183773,106.71962423199018,19.929915,256.815732
1755376967,428178821,1755376967,393276544,33.81595230102539,14.00549602508545,89.86083984375,1755376967,382926423,39.612323499844365,15.717162818997082,108.89851887688613,19.974007,276.789739
1755376967,928191654,1755376967,893520768,34.56009292602539,14.228487968444824,92.03022766113281,1755376967,882907706,40.41696334709238,15.962529081138307,111.04139365239882,19.968331,296.758071
1755376968,428189037,1755376968,393769600,35.31316375732422,14.452142715454102,94.20113372802734,1755376968,382912625,41.219883920024955,16.208501603089445,113.1471906570253,19.923032,316.681103
1755376968,928173482,1755376968,894019456,36.07353210449219,14.67931079864502,96.36790466308594,1755376968,882953666,42.02102330491135,16.455090363828788,115.21475394468547,19.842625,336.523728
1755376969,428160714,1755376969,394146176,36.83982849121094,14.909430503845215,98.53357696533203,1755376969,382928254,42.82013038784959,16.702246742592514,117.24248323224684,19.723118,356.246846
1755376969,928176621,1755376969,894265472,37.61464309692383,15.143107414245605,100.6889419555664,1755376969,882921397,43.617248996861036,16.950012914739105,119.22957333976737,19.571694,375.818540
1755376970,428172499,1755376970,394391424,38.40302658081055,15.380948066711426,102.83342742919922,1755376970,382894945,44.412241289234316,17.198374856640164,121.17479356262079,19.386062,395.204602
1755376970,928201726,1755376970,894515584,39.201534271240234,15.622862815856934,104.96065521240234,1755376970,882914554,45.2051284177482,17.44736816449791,123.07733970696151,19.172538,414.377140
1755376971,428179597,1755376971,394633856,40.010169982910156,15.868404388427734,107.07345581054688,1755376971,382939295,45.9957510092826,17.69697189500686,124.93600675212602,18.927270,433.304410
1755376971,928208531,1755376971,894882176,40.82600402832031,16.117307662963867,109.1670913696289,1755376971,882943742,46.783991324444486,17.94717775121687,126.74973954783789,18.654639,451.959049
1755376972,428187759,1755376972,395125888,41.65607833862305,16.372127532958984,111.25125122070312,1755376972,382923924,47.56975662006661,18.197985089650082,128.51758683234152,18.342074,470.301124
1755376972,928180461,1755376972,895373696,42.49334716796875,16.629905700683594,113.3036880493164,1755376972,882919470,48.353018320649454,18.449413739105864,130.2387808660713,18.012321,488.313445
1755376973,428181526,1755376973,395501184,43.340545654296875,16.89235496520996,115.3414306640625,1755376973,382916239,49.13367866086435,18.701461413622084,131.91243864843713,17.647424,505.960869
1755376973,928189334,1755376973,895628928,44.19429016113281,17.157114028930664,117.35847473144531,1755376973,883004377,49.911811803139244,18.95418160864373,133.53807818199735,17.253958,523.214827
1755376974,428185608,1755376974,397752576,45.06155776977539,17.427654266357422,119.3723373413086,1755376974,382912338,50.68685971366921,19.207421754316655,135.1139585999106,16.811015,540.025842
1755376974,928186189,1755376974,895993984,45.92597198486328,17.700620651245117,121.34439849853516,1755376974,882889635,51.459175553705705,19.461325821898512,136.64026658791698,16.360929,556.386772
1755376975,428185765,1755376975,396243328,46.805274963378906,17.98203468322754,123.2908706665039,1755376975,382901075,52.22860868463664,19.71587400615833,138.1161824828149,15.881077,572.267849
1755376975,928231678,1755376975,896371328,47.695594787597656,18.265995025634766,125.20021057128906,1755376975,882889210,52.994986798489066,19.9710385832185,139.54090838044866,15.383314,587.651163
1755376976,428169128,1755376976,398492672,48.60422134399414,18.5562686920166,127.09205627441406,1755376976,382965411,53.75839941332231,20.22687894978963,140.9141773249314,14.846128,602.497291
1755376976,928046661,1755376976,946853632,49.60717010498047,18.87825584411621,129.109619140625,1755376976,882959191,54.51849841559388,20.483307763174047,142.23498019568166,14.105759,616.603050
1755376977,428069091,1755376977,446980608,50.5311164855957,19.176700592041016,130.9072265625,1755376977,382944637,55.27532191311826,20.740366740162266,143.50305607052292,13.550183,630.153233
1755376977,928035907,1755376977,947209472,51.45836639404297,19.478673934936523,132.6688690185547,1755376977,882956216,56.028835718165055,20.99807372765172,144.71804772309142,12.976150,643.129383
1755376978,428182260,1755376978,449453440,52.400089263916016,19.788223266601562,134.40957641601562,1755376978,382959563,56.77891772968334,21.256416485379383,145.8795005202345,12.364825,655.494208
1755376978,928132974,1755376978,947576832,53.33881378173828,20.10028648376465,136.0938720703125,1755376978,882946294,57.52546117858404,21.515387551361172,146.98704085739632,11.755495,667.249702
1755376979,428071767,1755376979,447705344,54.28679275512695,20.418399810791016,137.7401580810547,1755376979,382936695,58.26842734560131,21.775002894541075,148.04044474601346,11.126082,678.375784
1755376979,928068145,1755376979,947956992,55.240108489990234,20.74009895324707,139.33689880371094,1755376979,882958029,59.007771480010405,22.035276608910188,149.0395178920775,10.488736,688.864520
1755376980,428183433,1755376980,398199680,56.098941802978516,21.030969619750977,140.7386932373047,1755376980,382900124,59.74325399414886,22.29615362496599,149.9838616193419,10.017726,698.882246
1755376980,928173495,1755376980,898449024,57.02920150756836,21.3482723236084,142.25286865234375,1755376980,882952332,60.47507473771387,22.557734623073912,150.87368892308666,9.362445,708.244692
1755376981,428179027,1755376981,398449024,57.927452087402344,21.6566162109375,143.7111053466797,1755376981,382906139,61.20285074128469,22.819912433729122,151.7085366162139,8.720115,716.964807
1755376981,928180975,1755376981,898575232,58.79310989379883,21.95583152770996,145.10601806640625,1755376981,882955048,61.926789812803065,23.082791614329572,152.48869060327735,8.099003,725.063809
1755376982,428168513,1755376982,398832768,59.62546157836914,22.247648239135742,146.4453887939453,1755376982,382959660,62.646602033232796,23.346296936243576,153.2139215824365,7.493154,732.556963
1755376982,928169374,1755376982,898963072,60.42874526977539,22.53268814086914,147.73184204101562,1755376982,882936526,63.3622436186952,23.610441669007336,153.88433043735853,6.900730,739.457693
1755376983,428048196,1755376983,449089280,61.27755355834961,22.83858299255371,149.07632446289062,1755376983,382952906,64.073730377506,23.87526154861608,154.50010658251318,6.189565,745.647259
1755376983,928055187,1755376983,949338624,62.0240592956543,23.108957290649414,150.2329559326172,1755376983,882941333,64.7808887705311,24.14072190051514,155.0613176324367,5.654885,751.302144
1755376984,428060426,1755376984,449466368,62.75223922729492,23.373794555664062,151.3231964111328,1755376984,382932446,65.48369341218552,24.40684293151204,155.56820870554174,5.152491,756.454634
1755376984,928081511,1755376984,949721600,63.4677619934082,23.636594772338867,152.3463592529297,1755376984,882933554,66.18207100933648,24.673626919606754,156.0210286522185,4.684667,761.139301
1755376985,428163000,1755376985,449981696,64.16986083984375,23.89802360534668,153.29794311523438,1755376985,382931430,66.87593351441356,24.941070248887932,156.42006507844815,4.261270,765.400571
1755376985,928086137,1755376985,950239744,64.86296081542969,24.158977508544922,154.1737823486328,1755376985,882948774,67.56524153120199,25.20918791310845,156.7656789948064,3.888855,769.289426
1755376986,428151929,1755376986,450486016,65.55226135253906,24.42146110534668,154.96669006347656,1755376986,382934296,68.24984068741401,25.47794984417972,157.05822268783675,3.573179,772.862605
1755376986,928172422,1755376986,950611200,66.24263000488281,24.68668556213379,155.67730712890625,1755376986,882945712,68.92974311264041,25.747390852106193,157.29817097444584,3.312533,776.175138
1755376987,428152571,1755376987,450864128,66.92787170410156,24.95117950439453,156.2952117919922,1755376987,382947693,69.60482755345026,26.017493179210987,157.4859865793469,3.117862,779.293000
1755376987,928036398,1755376987,951119616,67.60993194580078,25.21685791015625,156.8214874267578,1755376987,882947197,70.27503099720172,26.28826173126373,157.62220132192056,2.981912,782.274912
1755376988,428056900,1755376988,451373824,68.28723907470703,25.484373092651367,157.24703979492188,1755376988,382927272,70.9402578231701,26.559687943905658,157.70738565397653,2.899436,785.174348
1755376988,928322387,1755376988,953625472,68.96746063232422,25.755756378173828,157.5625,1755376988,882965295,71.60054162532722,26.83181613092509,157.742162869531,2.850140,788.024488
1755376989,428187597,1755376989,401866112,69.57036590576172,25.99708366394043,157.7251434326172,1755376989,382965718,72.25568801235477,27.10459671544416,157.7271814633237,2.904745,790.929233
1755376989,928184661,1755376989,902120832,70.23774719238281,26.266315460205078,157.74195861816406,1755376989,882965026,72.90566934068977,27.37804826036144,157.6631469012347,2.891361,793.820595
1755376990,428184981,1755376990,402367872,70.89862823486328,26.53429412841797,157.6490478515625,1755376990,382966720,73.55046117666993,27.652191173410216,157.55079266635602,2.879508,796.700102
1755376990,928202370,1755376990,904494592,71.56507873535156,26.8065128326416,157.51498413085938,1755376990,882932263,74.18982620655405,27.926954448467228,157.3909406110915,2.856585,799.556687
1755376991,428189341,1755376991,402750592,72.22172546386719,27.080001831054688,157.3590087890625,1755376991,382924264,74.82389761495597,28.202425477891097,157.18436875066294,2.839302,802.395989
1755376991,928206829,1755376991,903000960,72.87479400634766,27.3541316986084,157.1736297607422,1755376991,882889303,75.45249823356465,28.478558129348357,156.93196942410162,2.822639,805.218628
1755376992,428202233,1755376992,403134336,73.5208969116211,27.628244400024414,156.9532470703125,1755376992,382915358,76.07567678929007,28.755404768561704,156.6345898418605,2.810504,808.029132
1755376992,928215511,1755376992,903258240,74.16356658935547,27.903608322143555,156.69467163085938,1755376992,882897135,76.69322664521854,29.032904428367445,156.29323014380387,2.799222,810.828354
1755376993,428199498,1755376993,403383168,74.80039978027344,28.179340362548828,156.4008026123047,1755376993,382935108,77.3052136983593,29.311117559664694,155.90878744101792,2.792327,813.620682
1755376993,928194246,1755376993,903509888,75.43086242675781,28.455129623413086,156.0725555419922,1755376993,882898285,77.91140035325506,29.589966703499925,155.48237920138655,2.790920,816.411601
1755376994,428219939,1755376994,403637376,76.05645751953125,28.731531143188477,155.7094268798828,1755376994,382889436,78.51185956914162,29.869515743236036,155.0149553116876,2.793975,819.205577
1755376994,928182574,1755376994,903761536,76.67534637451172,29.009023666381836,155.31248474121094,1755376994,882908766,79.10652198956339,30.149763860998434,154.50758417174018,2.803528,822.009104
1755376995,428184239,1755376995,404014208,77.2867202758789,29.286331176757812,154.88169860839844,1755376995,382947414,79.69531425933943,30.430707912407627,153.96137158546478,2.820979,824.830084
1755376995,928194220,1755376995,904133248,77.88957214355469,29.564170837402344,154.4197235107422,1755376995,882917057,80.27806223996427,30.712295209045138,153.37756182972888,2.847661,827.677745
1755376996,428209317,1755376996,404258432,78.4880142211914,29.842378616333008,153.9259490966797,1755376996,382920694,80.85483393051629,30.994589223292422,152.7571922581028,2.880177,830.557922
1755376996,928258428,1755376996,904382592,79.08221435546875,30.12299346923828,153.40243530273438,1755376996,882908861,81.42550365301918,31.277559711480365,152.10151995708222,2.918289,833.476211
1755376997,428204162,1755376997,404509056,79.67523956298828,30.40488624572754,152.8463592529297,1755376997,382935576,81.99007569620943,31.561240155156895,151.41168445371363,2.958701,836.434912
1755376997,928200038,1755376997,904638848,80.26214599609375,30.687498092651367,152.2648162841797,1755376997,882913478,82.54838772522209,31.845580021384677,150.68905404300887,3.008502,839.443413
1755376998,428196503,1755376998,404883840,80.84226989746094,30.970638275146484,151.65472412109375,1755376998,382910145,83.1004631867381,32.13062214789119,149.93479064901783,3.066459,842.509872
1755376998,928192507,1755376998,905136256,81.41458129882812,31.25426483154297,151.01661682128906,1755376998,882916849,83.64622000503977,32.416355733391285,149.15021127284325,3.132752,845.642624
1755376999,428239483,1755376999,407385344,81.98441314697266,31.540325164794922,150.3447265625,1755376999,382884849,84.18555453018226,32.70275739271759,148.33669495674133,3.198197,848.840821
1755376999,928175613,1755376999,905632640,82.54114532470703,31.821880340576172,149.65322875976562,1755376999,882903212,84.71850765186275,32.989880376518286,147.49542102516148,3.280437,852.121258
1755377000,428055927,1755377000,456132352,83.14723205566406,32.13345718383789,148.8603973388672,1755377000,382915594,85.24495951102853,33.27769212697859,146.62783132865346,3.270181,855.391439
1755377000,928177089,1755377000,956359424,83.68994140625,32.41746139526367,148.11465454101562,1755377000,882940898,85.76487178976595,33.56620325984008,145.73526050820865,3.359533,858.750972
1755377001,428184308,1755377001,408480512,84.17764282226562,32.67711639404297,147.4143829345703,1755377001,382937755,86.27813980221512,33.85538690601841,144.81917576287063,3.540552,862.291524
1755377001,928192701,1755377001,906602112,84.70684051513672,32.96234130859375,146.6212158203125,1755377001,882948662,86.78476600214499,34.145275827974814,143.88087082638907,3.636839,865.928363
1755377002,428172191,1755377002,456830464,85.28629302978516,33.27949523925781,145.7239532470703,1755377002,382927132,87.28463715441225,34.435836620993314,142.9218638338951,3.630731,869.559094
1755377002,928169932,1755377002,956952832,85.80601501464844,33.56829833984375,144.89004516601562,1755377002,882991628,87.77782347282825,34.72714245901983,141.9433310030562,3.730157,873.289252
1755377003,428189215,1755377003,409080832,86.2725830078125,33.83087158203125,144.1137237548828,1755377003,382928467,88.26406026931632,35.01906814064748,140.94712458837895,3.924939,877.214190
1755377003,928167529,1755377003,907319936,86.78060913085938,34.119319915771484,143.23997497558594,1755377003,882942147,88.74349373802977,35.31173243856388,139.93427965428086,4.025219,881.239409
1755377004,428194101,1755377004,407561600,87.2819595336914,34.40919876098633,142.3433837890625,1755377004,382948554,89.21599480203571,35.6050894089488,138.9063967711407,4.121107,885.360516
1755377004,928185709,1755377004,907808896,87.77494812011719,34.700870513916016,141.4304656982422,1755377004,882962409,89.6815214752249,35.8991448585614,137.86490531363643,4.217120,889.577637
1755377005,428193554,1755377005,408056448,88.26164245605469,34.994483947753906,140.49769592285156,1755377005,382953244,90.140000968252,36.19388449547559,136.8113096327901,4.307695,893.885332
1755377005,928170802,1755377005,958180352,88.79010009765625,35.31606674194336,139.451904296875,1755377005,882927125,90.59138314565966,36.48930740095845,135.74706936799103,4.283330,898.168662
1755377006,428170399,1755377006,458427136,89.26062774658203,35.608070373535156,138.48397827148438,1755377006,382949226,91.03568304892947,36.78545587746696,134.67348735422337,4.365421,902.534083
1755377006,928193048,1755377006,958656512,89.72369384765625,35.9005126953125,137.50523376464844,1755377006,882929827,91.47277369563393,37.08227750918334,133.59220825100718,4.446079,906.980161
1755377007,428212741,1755377007,408783488,90.1374282836914,36.1656379699707,136.6114959716797,1755377007,382939421,91.90266854901245,37.37981338583458,132.5045350230417,4.632216,911.612377
1755377007,928240408,1755377007,908905856,90.59136962890625,36.46190643310547,135.61122131347656,1755377007,882967950,92.32530971958299,37.67805640374044,131.41194175476252,4.703139,916.315516
1755377008,428203404,1755377008,409026688,91.0377197265625,36.75910186767578,134.6036834716797,1755377008,382936808,92.74058918171448,37.97696183010379,130.31603554203488,4.771464,921.086981
1755377008,928182354,1755377008,909146496,91.47615051269531,37.05549621582031,133.58314514160156,1755377008,882941498,93.14854078363955,38.276585897420986,129.21804630732262,4.831360,925.918340
1755377009,428216297,1755377009,409272448,91.90644073486328,37.35296630859375,132.55287170410156,1755377009,382955477,93.54909994501512,38.576914365632945,128.11945398719078,4.883807,930.802147
1755377009,928188903,1755377009,909394816,92.33145141601562,37.65301513671875,131.50851440429688,1755377009,882921701,93.94217124742711,38.877907252630635,127.02182210424075,4.921909,935.724056
1755377010,428183141,1755377010,409519744,92.74746704101562,37.953338623046875,130.45941162109375,1755377010,382917933,94.32778368628604,39.17961880696966,125.9263588312552,4.954769,940.678825
1755377010,928185070,1755377010,909644160,93.15460205078125,38.25283432006836,129.4021759033203,1755377010,882942817,94.70589024853516,39.48204600448241,124.83446962689887,4.978092,945.656917
1755377011,428227726,1755377011,409890688,93.5539779663086,38.5511360168457,128.34359741210938,1755377011,382906059,95.07637805447818,39.785130343038745,123.74774406016778,4.996230,950.653146
1755377011,928190192,1755377011,912132096,93.9492416381836,38.853206634521484,127.2750244140625,1755377011,882916325,95.4392936487556,40.08894212161741,122.66729313653201,4.997848,955.650994
1755377012,428183355,1755377012,410384256,94.33415985107422,39.153499603271484,126.21578216552734,1755377012,382977626,95.79459803910845,40.393483500760226,121.59445855064826,5.002706,960.653701
1755377012,928181517,1755377012,910509696,94.71358489990234,39.455841064453125,125.15254211425781,1755377012,882976440,96.14216989314748,40.698682908151454,120.53082008732736,4.994580,965.648280
1755377013,428244088,1755377013,410734976,95.0850830078125,39.75799560546875,124.09111785888672,1755377013,382955155,96.48200376385805,41.00456636630118,119.47759204668722,4.978950,970.627231
1755377013,928371679,1755377013,912989440,95.45207977294922,40.06443786621094,123.02296447753906,1755377013,882925031,96.8140732844982,41.31114225356991,118.43603109206487,4.944619,975.571850
1755377014,428252580,1755377014,411118976,95.80702209472656,40.369266510009766,121.9721450805664,1755377014,382942859,97.13837817635327,41.61844408266377,117.40728933177081,4.916387,980.488237
1755377014,928190396,1755377014,911242624,96.15437316894531,40.675689697265625,120.91867065429688,1755377014,882962378,97.45485266968804,41.92644359823676,116.39270415509253,4.872372,985.360608
1755377015,428188793,1755377015,411367808,96.49454498291016,40.98194885253906,119.86902618408203,1755377015,382938545,97.76343383377603,42.23511137705239,115.39358560969002,4.817682,990.178291
1755377015,928200275,1755377015,913619968,96.82998657226562,41.290470123291016,118.81599426269531,1755377015,882941490,98.06413516173659,42.54449346910562,114.41097277552895,4.743407,994.921698
1755377016,428167994,1755377016,411871872,97.1520004272461,41.595184326171875,117.7774658203125,1755377016,382923848,98.35689174290897,42.85455623436383,113.4461341247314,4.668856,999.590554
1755377016,928177499,1755377016,912126336,97.46675872802734,41.901451110839844,116.74578857421875,1755377016,882945406,98.64171124139229,43.16534136416949,112.50007379759712,4.583015,1004.173569
1755377017,428153644,1755377017,412249728,97.77399444580078,42.20856475830078,115.72793579101562,1755377017,382944611,98.9185226646235,43.47680495847065,111.57402924895433,4.491472,1008.665040
1755377017,928185934,1755377017,912376448,98.07705688476562,42.517887115478516,114.71878814697266,1755377017,882919484,99.18729913606555,43.78894844661107,110.66906451502,4.387311,1013.052351
1755377018,428211552,1755377018,412503680,98.37261962890625,42.82817077636719,113.72300720214844,1755377018,382933170,99.44804603847865,44.10181177786605,109.78610454443165,4.275267,1017.327618
1755377018,928265440,1755377018,912753536,98.66016387939453,43.140323638916016,112.74711608886719,1755377018,882956272,99.70071661119096,44.41537356485726,108.92621962461025,4.160259,1021.487877
1755377019,428182564,1755377019,412989568,98.93783569335938,43.45288848876953,111.78563690185547,1755377019,382915714,99.9452484494115,44.729587753986365,108.09050372974855,4.037184,1025.525060
1755377019,928178904,1755377019,913110400,99.20682525634766,43.766014099121094,110.83763122558594,1755377019,882985301,100.18170119125797,45.044565865634894,107.27960104474025,3.904441,1029.429501
1755377020,428170935,1755377020,413110400,99.46769714355469,44.080841064453125,109.90486145019531,1755377020,382951986,100.40994586618427,45.36016966837961,106.49476869819537,3.762080,1033.191581
1755377020,928171331,1755377020,913236096,99.71987915039062,44.39430236816406,108.993408203125,1755377020,882947312,100.63002070069369,45.6764825631261,105.73665639334855,3.616459,1036.808040
1755377021,428189699,1755377021,413486720,99.96343231201172,44.707618713378906,108.10395812988281,1755377021,382912813,100.84187692656275,45.99346775024105,105.00617763886643,3.467177,1040.275217
1755377021,928168433,1755377021,913613184,100.19914245605469,45.021331787109375,107.23800659179688,1755377021,882895855,101.04551153611632,46.311153600409895,104.30405257019724,3.314825,1043.590042
1755377022,428164568,1755377022,413856128,100.42790985107422,45.33645248413086,106.39031982421875,1755377022,382927046,101.2409157556105,46.62956262924439,103.6309848044769,3.153893,1046.743935
1755377022,928180169,1755377022,914105216,100.64813232421875,45.65106201171875,105.56942749023438,1755377022,882940623,101.42803934877365,46.94864776475829,102.98778363210185,2.992803,1049.736738
1755377023,428176272,1755377023,414355072,100.8608627319336,45.96773910522461,104.7804946899414,1755377023,382913665,101.6068556433593,47.268394169559954,102.37513872194667,2.834422,1052.571160
1755377023,928204846,1755377023,914592384,101.06507873535156,46.284637451171875,104.02284240722656,1755377023,882919406,101.77737326137486,47.588850338974225,101.79358032405598,2.679169,1055.250328
1755377024,428197863,1755377024,416845824,101.26203155517578,46.6067008972168,103.2872085571289,1755377024,382975442,101.93957739674627,47.91002607314819,101.24368059465965,2.516691,1057.767019
1755377024,928163366,1755377024,915102848,101.44970703125,46.92714309692383,102.58898162841797,1755377024,882930595,102.09340225845908,48.23182185403682,100.72614331324291,2.363619,1060.130638
1755377025,428177639,1755377025,415223168,101.62916564941406,47.24817657470703,101.92093658447266,1755377025,382930541,102.23887827118986,48.55433157616966,100.24130666452787,2.213357,1062.343995
1755377025,928163601,1755377025,915346816,101.79972076416016,47.566707611083984,101.28890991210938,1755377025,882925160,102.37597482380366,48.87752309970517,99.7896740706758,2.073165,1064.417160
1755377026,428167695,1755377026,417475072,101.96234130859375,47.88867950439453,100.68801879882812,1755377026,382933952,102.50468193643954,49.20140681454993,99.37163853394195,1.936555,1066.353715
1755377026,928177950,1755377026,915603072,102.11614227294922,48.208499908447266,100.13014221191406,1755377026,882929902,102.62497881416961,49.52596358681592,98.98758629273689,1.816607,1068.170321
1755377027,428178198,1755377027,415729280,102.26199340820312,48.52962112426758,99.60945129394531,1755377027,382923667,102.7368568829349,49.85120251176075,98.63782158389478,1.707670,1069.877991
1755377027,928187209,1755377027,915856000,102.39932250976562,48.85145950317383,99.13240051269531,1755377027,882935372,102.84030731724292,50.1771322743334,98.32260772404408,1.614819,1071.492810
1755377028,428185461,1755377028,417982464,102.52870178222656,49.178977966308594,98.69682312011719,1755377028,382918969,102.9353168575468,50.50374586606621,98.04217895803845,1.532614,1073.025424
1755377028,928171506,1755377028,916107904,102.64879608154297,49.504150390625,98.31193542480469,1755377028,882952684,103.02187169144878,50.83102776391746,97.79672673964973,1.471472,1074.496896
1755377029,428156593,1755377029,416237952,102.76183319091797,49.83034896850586,97.97346496582031,1755377029,382944714,103.09996407737674,51.15897842073541,97.5863805693318,1.424578,1075.921474
1755377029,928192136,1755377029,916363136,102.86607360839844,50.15614700317383,97.68527221679688,1755377029,882952236,103.16959117422684,51.48761898087626,97.41121430755565,1.392856,1077.314330
1755377030,428152057,1755377030,416486528,102.96119689941406,50.4815559387207,97.44911193847656,1755377030,382937255,103.23073967468146,51.81691792858557,97.2712845022337,1.373851,1078.688181
1755377030,928189250,1755377030,916836224,103.04800415039062,50.809730529785156,97.26911163330078,1755377030,882936867,103.2834089488585,52.14690206413742,97.16657274596956,1.361601,1080.049782
1755377031,428181217,1755377031,416958592,103.1247329711914,51.13966369628906,97.14874267578125,1755377031,383054591,103.32760173207208,52.477642265375835,97.09701923964786,1.354259,1081.404041
1755377031,928181680,1755377031,917199488,103.1943130493164,51.46749496459961,97.08429718017578,1755377031,882957400,103.363284983327,52.8089004664964,97.06256407648364,1.352181,1082.756222
1755377032,428182931,1755377032,417422976,103.255859375,51.79582214355469,97.06082153320312,1755377032,382962450,103.39048324949466,53.14090461416801,97.06302626178285,1.351804,1084.108027
1755377032,928314256,1755377032,917547648,103.30884552001953,52.123085021972656,97.07350158691406,1755377032,882941440,103.40918433364686,53.47355785536497,97.09822367734637,1.354421,1085.462447
1755377033,428162752,1755377033,417800576,103.3536605834961,52.45223617553711,97.12126922607422,1755377033,382936160,103.41938912842744,53.80689284576563,97.16792229470558,1.357052,1086.819500
1755377033,928181703,1755377033,918052992,103.39068603515625,52.78352737426758,97.1975326538086,1755377033,882934841,103.42109637375302,54.140895933960834,97.27183967046349,1.359741,1088.179241
1755377034,428180477,1755377034,418175360,103.41919708251953,53.11704635620117,97.30360412597656,1755377034,382930713,103.41430674778108,54.47556212752254,97.40964705044226,1.362657,1089.541898
1755377034,928250003,1755377034,918401664,103.43834686279297,53.45322799682617,97.43901062011719,1755377034,882952057,103.39902094411141,54.81090910555996,97.58098202968299,1.365650,1090.907548
1755377035,428188612,1755377035,418655104,103.44799041748047,53.78805160522461,97.60187530517578,1755377035,382904975,103.37524541720266,55.14687400227301,97.7853922247916,1.373087,1092.280635
1755377035,928185619,1755377035,918905472,103.44926452636719,54.122398376464844,97.7916030883789,1755377035,882912400,103.34297836517713,55.483535970705404,98.02246445263,1.384662,1093.665298
1755377036,428155727,1755377036,419139968,103.44200897216797,54.45652389526367,98.008544921875,1755377036,382940218,103.30222413749388,55.82087478746591,98.29168857329003,1.400415,1095.065713
1755377036,928185073,1755377036,922372160,103.42760467529297,54.79285430908203,98.25511932373047,1755377036,882948579,103.25299220977607,56.158855857045296,98.59249080716518,1.417840,1096.483553
1755377037,428198187,1755377037,419613568,103.40650939941406,55.125633239746094,98.5258560180664,1755377037,382924375,103.19529151656768,56.49747080555974,98.92426990206339,1.444052,1097.927604
1755377037,928179151,1755377037,919868544,103.37591552734375,55.46049880981445,98.82327270507812,1755377037,882939713,103.129120087364,56.83677174597305,99.28644790862492,1.472945,1099.400549
1755377038,428182317,1755377038,420087680,103.33477020263672,55.79679870605469,99.14583587646484,1755377038,382928681,103.05449623592492,57.17670474045002,99.67829756044654,1.505394,1100.905943
1755377038,928173528,1755377038,922338560,103.28465270996094,56.138431549072266,99.49642944335938,1755377038,882962172,102.97141675242105,57.517323687835756,100.09917139464257,1.537127,1102.443069
1755377039,428189645,1755377039,420583552,103.22490692138672,56.47673034667969,99.86734771728516,1755377039,382917146,102.87991457023757,57.858537764271034,100.54820420125519,1.578600,1104.021669
1755377039,928153714,1755377039,920832896,103.15730285644531,56.8172607421875,100.26310729980469,1755377039,882947519,102.77997213678944,58.20045354267687,101.0247469196107,1.623483,1105.645151
1755377040,428182512,1755377040,421086592,103.08164978027344,57.15671157836914,100.68304443359375,1755377040,382943988,102.67162311731076,58.54299282865868,101.52786734214095,1.674402,1107.319554
1755377040,928115321,1755377040,923327232,102.99829864501953,57.49966812133789,101.13052368164062,1755377040,882972696,102.55486569010617,58.88620179939463,102.05676984016212,1.725410,1109.044964
1755377041,428179767,1755377041,421577856,102.90837097167969,57.83919906616211,101.59768676757812,1755377041,382930391,102.42974069882382,59.230003558336776,102.61043390975047,1.785800,1110.830764
1755377041,928208356,1755377041,921826176,102.80876922607422,58.180755615234375,102.08819580078125,1755377041,882964851,102.29622494519107,59.574501800748095,103.18809277812511,1.847973,1112.678737
1755377042,428200230,1755377042,422067072,102.70079040527344,58.52124786376953,102.59866333007812,1755377042,382947498,102.15436875659181,59.91960513391379,103.78862728947502,1.915723,1114.594459
1755377042,928205175,1755377042,924315136,102.58430480957031,58.865478515625,103.1352310180664,1755377042,882946192,102.00417214394177,60.26535596389907,104.41110815655358,1.980927,1116.575387
1755377043,428205191,1755377043,422556544,102.4600830078125,59.20697021484375,103.68851470947266,1755377043,383045504,101.84562267284802,60.61181650580099,105.05462213625836,2.053632,1118.629019
1755377043,928155452,1755377043,922673280,102.32789611816406,59.55086135864258,104.26286315917969,1755377043,882943638,101.6788397294621,60.95877116444028,105.71770043474524,2.126038,1120.755056
1755377044,428174323,1755377044,422795904,102.18696594238281,59.89585876464844,104.85242462158203,1755377044,382941310,101.50374523065668,61.30642769652489,106.399642284098,2.202357,1122.957414
1755377044,928200717,1755377044,922911616,102.03644561767578,60.24240493774414,105.46273040771484,1755377044,882966992,101.32037885358375,61.65474103269922,107.09924604878799,2.277197,1125.234611
1755377045,428193527,1755377045,423033216,101.8774185180664,60.59005355834961,106.09210205078125,1755377045,382929247,101.12880564443195,62.003629624371655,107.81519881786204,2.351102,1127.585713
1755377045,928206672,1755377045,923284608,101.70956420898438,60.93798828125,106.73832702636719,1755377045,882907938,100.92901075516338,62.35316296863412,108.5464680423753,2.425151,1130.010864
1755377046,428209531,1755377046,423410048,101.5335922241211,61.28841018676758,107.4005126953125,1755377046,382930153,100.72100908757388,62.70335238976159,109.29188295863234,2.497926,1132.508790
1755377046,928186328,1755377046,923532672,101.34999084472656,61.63864517211914,108.07864379882812,1755377046,882950320,100.50484470897213,63.05416240114086,110.05014904574483,2.569979,1135.078769
1755377047,428178595,1755377047,423658112,101.15872955322266,61.98853302001953,108.77210998535156,1755377047,382922392,100.28056503546185,63.405558425094654,110.81994765191553,2.640601,1137.719370
1755377047,928184516,1755377047,923902080,100.96136474609375,62.335784912109375,109.48001098632812,1755377047,882943611,100.04814783538883,63.75761534091565,111.60018052586378,2.711215,1140.430585
1755377048,428177373,1755377048,424140160,100.755126953125,62.682640075683594,110.20093536376953,1755377048,382964040,99.80765280776394,64.11028147187903,112.3894528665962,2.779474,1143.210059
1755377048,928174034,1755377048,924385664,100.54043579101562,63.03159713745117,110.93163299560547,1755377048,882986530,99.55909942896143,64.46356883455742,113.18649232323246,2.845691,1146.055750
1755377049,428183519,1755377049,424609408,100.31437683105469,63.38544845581055,111.67486572265625,1755377049,382926091,99.30256750170653,64.81740709942099,113.98982383515781,2.904014,1148.959763
1755377049,928177384,1755377049,924733568,100.07981872558594,63.73931884765625,112.4230728149414,1755377049,882923994,99.03801129746212,65.17190071286466,114.79835449892119,2.963042,1151.922805
1755377050,428186054,1755377050,424980096,99.83769226074219,64.0929946899414,113.18220520019531,1755377050,383023936,98.76543617953432,65.5270780600039,115.61080750703039,3.017356,1154.940161
1755377050,928196920,1755377050,925222528,99.59011840820312,64.44526672363281,113.94971466064453,1755377050,882955743,98.48502929616458,65.88273802017696,116.42536777141856,3.068616,1158.008777
1755377051,428187311,1755377051,427466496,99.33234405517578,64.80097961425781,114.73171997070312,1755377051,382954821,98.19669109927975,66.23905066771718,117.24105971833939,3.107176,1161.115953
1755377051,928185843,1755377051,925589376,99.06961059570312,65.1551284790039,115.51313018798828,1755377051,882929959,97.90050794010678,66.59594892446003,118.05635642209957,3.148137,1164.264090
1755377052,428201008,1755377052,425712256,98.79730987548828,65.51050567626953,116.30406188964844,1755377052,382926823,97.59648975623462,66.9534618194365,118.86994364383361,3.179283,1167.443373
1755377052,928192498,1755377052,925833088,98.51882934570312,65.86347961425781,117.09746551513672,1755377052,882989215,97.28464095975906,67.3116228388482,119.68051019570017,3.208186,1170.651559
1755377053,428168678,1755377053,428077824,98.23033142089844,66.22013092041016,117.8990249633789,1755377053,383021110,96.96506379679568,67.67035475046248,120.48649021253786,3.224752,1173.876311
1755377053,928198919,1755377053,926315136,97.93423461914062,66.57473754882812,118.69303131103516,1755377053,883039809,96.63777985796024,68.02967547931456,121.28653164776583,3.244054,1177.120365
1755377054,428174096,1755377054,426560640,97.62899780273438,66.9333724975586,119.4893569946289,1755377054,382953743,96.30290816342747,68.38949863574064,122.07905167657444,3.253511,1180.373876
1755377054,928182546,1755377054,926678400,97.31751251220703,67.29261016845703,120.2835693359375,1755377054,882934701,95.9603574540828,68.74996468181311,122.86296816547008,3.258688,1183.632565
1755377055,428175927,1755377055,428925952,96.99595642089844,67.6535873413086,121.08269500732422,1755377055,382971735,95.61017893399305,69.11105818556156,123.63685270872097,3.250896,1186.883460
1755377055,928247772,1755377055,927156608,96.66979217529297,68.0118408203125,121.86705780029297,1755377055,883005087,95.25245814661773,69.47273132873526,124.39921383621827,3.248823,1190.132283
1755377056,428186380,1755377056,427282560,96.33405303955078,68.3729476928711,122.64957427978516,1755377056,383113394,94.88717456515117,69.83504487824105,125.14879443605543,3.236864,1193.369147
1755377056,928182944,1755377056,927409536,95.99149322509766,68.7338638305664,123.42687225341797,1755377056,882956772,94.51464100976013,70.19773430680569,125.88368150006843,3.218683,1196.587830
1755377057,428168439,1755377057,427655296,95.64114379882812,69.09606170654297,124.2013931274414,1755377057,382941248,94.1345882405543,70.56110617913512,126.60313659329893,3.191306,1199.779136
1755377057,928158953,1755377057,927776384,95.28411865234375,69.45684051513672,124.96980285644531,1755377057,882941452,93.74715917079402,70.92506257249015,127.30560185813566,3.158145,1202.937281
1755377058,428161542,1755377058,428016256,94.92119598388672,69.81539916992188,125.72689056396484,1755377058,382945554,93.35240778855504,71.28959434902751,127.98971413520012,3.123254,1206.060536
1755377058,928182492,1755377058,928253312,94.54989624023438,70.1757583618164,126.46993255615234,1755377058,882938269,92.95040001484917,71.65468289668699,128.65410794856606,3.084838,1209.145374
1755377059,428236744,1755377059,428490880,94.16991424560547,70.53850555419922,127.19904327392578,1755377059,382922192,92.54117898882272,72.02033164070284,129.297474676205,3.041710,1212.187084
1755377059,928097947,1755377059,928611712,93.7821044921875,70.90544891357422,127.91422271728516,1755377059,882991314,92.12471722161285,72.38660606065037,129.91862396815696,2.993056,1215.180140
1755377060,428158570,1755377060,428966272,93.38645935058594,71.27262115478516,128.6094970703125,1755377060,382941572,91.70123825558623,72.75335254146457,130.51601553289984,2.944036,1218.124176
1755377060,928048894,1755377060,929207424,92.98330688476562,71.63928985595703,129.2857666015625,1755377060,882956500,91.27063689610439,73.12070558474537,131.0886093575793,2.894490,1221.018666
1755377061,428153721,1755377061,429453952,92.57291412353516,72.00542449951172,129.9431915283203,1755377061,382945660,90.83305137812326,73.48858930429668,131.63504238682037,2.844162,1223.862829
1755377061,928280737,1755377061,929690240,92.15336608886719,72.37207794189453,130.58349609375,1755377061,882980874,90.38846152213007,73.8570630820635,132.15418119215278,2.790541,1226.653370
1755377062,428175279,1755377062,429808000,91.73011016845703,72.7386474609375,131.1984100341797,1755377062,382891757,89.93708332423385,74.225991901099,132.6446460430991,2.742032,1229.395402
1755377062,928176178,1755377062,930052480,91.30175018310547,73.10574340820312,131.78773498535156,1755377062,882905151,89.47876375208885,74.59554412572348,133.10549177770673,2.698012,1232.093413
1755377063,428181197,1755377063,430172032,90.8653335571289,73.47207641601562,132.34791564941406,1755377063,382914718,89.01365840369465,74.96563586310072,133.53546018648652,2.658887,1234.752301
1755377063,928116441,1755377063,932294912,90.41890716552734,73.84233856201172,132.88372802734375,1755377063,882918432,88.54182327874109,75.33626584172337,133.93343392186833,2.618615,1237.370916
1755377064,428265796,1755377064,430534528,89.97122192382812,74.20767974853516,133.38311767578125,1755377064,382929689,88.0633076029864,75.70743862063686,134.2983301099909,2.593651,1239.964567
1755377064,928175504,1755377064,930655616,89.51373291015625,74.57640838623047,133.84689331054688,1755377064,882947037,87.57817613281269,76.07914724945303,134.62908427663794,2.572242,1242.536809
1755377065,428162491,1755377065,430891136,89.04844665527344,74.94728088378906,134.2711944580078,1755377065,382916823,87.08653306710008,76.45135595658273,134.92464414312428,2.557018,1245.093828
1755377065,928161189,1755377065,933126912,88.57286071777344,75.32168579101562,134.659423828125,1755377065,882926282,86.58836124076751,76.82412143172283,135.1840636702987,2.543776,1247.637604
1755377066,428168166,1755377066,431362944,88.09571838378906,75.69326782226562,135.0010528564453,1755377066,382920385,86.08377731540494,77.19739986257832,135.40635301495269,2.544521,1250.182125
1755377066,928199164,1755377066,931595392,87.61054229736328,76.06529235839844,135.2977752685547,1755377066,882909455,85.5728315557854,77.5711981973697,135.59059722117826,2.550640,1252.732765
1755377067,428175851,1755377067,431715968,87.11993408203125,76.43565368652344,135.54351806640625,1755377067,382962525,85.05551464742965,77.9455664639713,135.73592885376416,2.564895,1255.297660
1755377067,928210866,1755377067,933834496,86.6179428100586,76.80977630615234,135.734619140625,1755377067,882923392,84.53205772491256,78.3203818205326,135.84145660646033,2.577646,1257.875306
1755377068,428171713,1755377068,431954048,86.11430358886719,77.18143463134766,135.85987854003906,1755377068,382916449,84.0023971683828,78.69573477691777,135.90640566324245,2.599119,1260.474425
1755377068,928187932,1755377068,932194176,85.60399627685547,77.55267333984375,135.91976928710938,1755377068,882926066,83.46661492371862,79.07161105442846,135.93000007492216,2.622151,1263.096575
1755377069,428174560,1755377069,432314752,85.08843231201172,77.92620849609375,135.9293212890625,1755377069,382878336,82.92486029989595,79.44795085323692,135.9115150108375,2.645196,1265.741771
1755377069,928186611,1755377069,932436096,84.5637435913086,78.30310821533203,135.8997802734375,1755377069,882944426,82.3770089775314,79.82488448803969,135.85025214673414,2.664595,1268.406366
1755377070,428175598,1755377070,432673664,84.0325698852539,78.68131256103516,135.8311309814453,1755377070,382881228,81.82341034580716,80.20221649627791,135.74560453526007,2.683440,1271.089806
1755377070,928202235,1755377070,932786816,83.49659729003906,79.05807495117188,135.7206573486328,1755377070,882904448,81.2638835527222,80.58011549456316,135.59693418982837,2.704982,1273.794787
1755377071,428195278,1755377071,432906880,82.95491790771484,79.43360900878906,135.57241821289062,1755377071,382919764,80.69860948399318,80.95850345924195,135.40370863983756,2.728497,1276.523284
1755377071,928173333,1755377071,933145216,82.40778350830078,79.8111801147461,135.39015197753906,1755377071,882898300,80.12769347417964,81.33735546144233,135.165440066768,2.752911,1279.276195
1755377072,428164744,1755377072,433381248,81.85269927978516,80.18997192382812,135.1728973388672,1755377072,382917934,79.55111895888518,81.71672771135769,134.88163239032366,2.777245,1282.053440
1755377072,928171683,1755377072,933619328,81.29210662841797,80.56714630126953,134.920654296875,1755377072,882935894,78.96900953377903,82.0965839084049,134.55189452841773,2.805698,1284.859138
1755377073,428175718,1755377073,383734016,80.7847900390625,80.90544128417969,134.66224670410156,1755377073,382920326,78.38147919972909,82.4768955453645,134.17589138013759,2.912372,1287.771510
1755377073,928180813,1755377073,933973376,80.16049194335938,81.31954956054688,134.3072509765625,1755377073,882916402,77.7885462458488,82.8576966514585,133.75327584678817,2.880783,1290.652294
1755377074,428208420,1755377074,434210944,79.58340454101562,81.70023345947266,133.945068359375,1755377074,382913483,77.19030650013283,83.23897139369664,133.28379669107727,2.920944,1293.573237
1755377074,928176874,1755377074,884449792,79.05877685546875,82.04251861572266,133.58953857421875,1755377074,882932714,76.58680444570999,83.62073718755904,132.76721182707095,3.045922,1296.619160
1755377075,428217825,1755377075,384663040,78.46966552734375,82.42430114746094,133.16050720214844,1755377075,383272275,75.97774768325505,84.00322216329313,132.20299594380174,3.101528,1299.720687
1755377075,928177863,1755377075,884781312,77.87559509277344,82.8051986694336,132.69577026367188,1755377075,882908706,75.36449795009102,84.38562063855646,131.59227595771247,3.165603,1302.886290
1755377076,428186324,1755377076,386904448,77.2725830078125,83.18853759765625,132.18997192382812,1755377076,382965508,74.74572460140482,84.76881108477947,130.93362378827985,3.234299,1306.120589
1755377076,928234146,1755377076,885024256,76.67127990722656,83.57010650634766,131.655517578125,1755377076,882921287,74.1221624307216,85.15238238439176,130.2276856379305,3.322695,1309.443284
1755377077,428174009,1755377077,385143296,76.06409454345703,83.95355987548828,131.08181762695312,1755377077,382904105,73.49373615882192,85.53642811356897,129.4743372347892,3.419972,1312.863256
1755377077,928248695,1755377077,885261056,75.45185852050781,84.33587646484375,130.47071838378906,1755377077,882923549,72.8605113117971,85.92095459517019,128.67364658335015,3.529450,1316.392706
1755377078,428187480,1755377078,387507840,74.83038330078125,84.72151947021484,129.81692504882812,1755377078,383045496,72.22247014267106,86.30601850073634,127.8256156673741,3.643784,1320.036490
1755377078,928182689,1755377078,885626368,74.21189880371094,85.10193634033203,129.13201904296875,1755377078,882924818,71.58018685999224,86.6913214983638,126.93111586465017,3.781009,1323.817499
1755377079,428197787,1755377079,385744384,73.58668518066406,85.48426818847656,128.40850830078125,1755377079,382948395,70.9331757867455,87.07720286242143,125.98961756304207,3.928051,1327.745550
1755377079,928197242,1755377079,885862144,72.95410919189453,85.86859893798828,127.6504135131836,1755377079,882910178,70.28189292640147,87.46343761460714,125.00195827635427,4.086388,1331.831938
1755377080,428174738,1755377080,386104576,72.31415557861328,86.25306701660156,126.85496520996094,1755377080,382895339,69.62622228373215,87.85014256442471,123.96819809291944,4.255474,1336.087412
1755377080,928181717,1755377080,886340352,71.66893005371094,86.64056396484375,126.02355194091797,1755377080,882930411,68.9662412284404,88.23731922828937,122.88872211500967,4.436363,1340.523774
1755377081,428175731,1755377081,436568960,70.95547485351562,87.06651306152344,125.07310485839844,1755377081,382918561,68.30217608218987,88.62488276890383,121.76422133629849,4.518542,1345.042316
1755377081,928191384,1755377081,936688000,70.30522918701172,87.45265197753906,124.17134857177734,1755377081,882919026,67.63403048750573,89.01287940235645,120.59507335866599,4.728568,1349.770884
1755377082,428202949,1755377082,436928896,69.64791107177734,87.83946990966797,123.22789764404297,1755377082,382907409,66.96192377423318,89.40128808892584,119.3818993848206,4.944240,1354.715124
1755377082,928191618,1755377082,937053568,68.98651885986328,88.22770690917969,122.24812316894531,1755377082,882914112,66.28591143816425,89.79012516692629,118.12525715414594,5.170344,1359.885468
1755377083,428167566,1755377083,437296000,68.32205200195312,88.61434173583984,121.23461151123047,1755377083,382919275,65.60610745600496,90.1793736131506,116.8258604072085,5.409507,1365.294975
1755377083,928186843,1755377083,937414272,67.65567779541016,89.0015869140625,120.18737030029297,1755377083,882904458,64.92263164901982,90.56901372563584,115.48448891018265,5.660694,1370.955669
1755377084,428180037,1755377084,437535360,66.98430633544922,89.39046478271484,119.10496520996094,1755377084,382916231,64.23550727743387,90.95908117163697,114.10178036888793,5.920162,1376.875831
1755377084,928197369,1755377084,937781120,66.31077575683594,89.77960968017578,117.9841079711914,1755377084,882901893,63.544902173245646,91.34952979806587,112.67870895833816,6.185626,1383.061457
1755377085,428159039,1755377085,438020224,65.63347625732422,90.1666259765625,116.83118438720703,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,6.293279,1389.354736
1755377085,928210193,1755377085,938257536,64.9587631225586,90.55022430419922,115.68649291992188,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,4.915614,1394.270350
1755377086,428197180,1755377086,388372992,64.39718627929688,90.86781311035156,114.75006103515625,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,3.786892,1398.057242
1755377086,928200194,1755377086,890492288,63.859806060791016,91.17000579833984,113.8377685546875,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,2.698882,1400.756123
1755377087,428182540,1755377087,388613376,63.43915557861328,91.40702819824219,113.07511138916016,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,1.812011,1402.568135
1755377087,928195002,1755377087,888733696,63.12775421142578,91.58162689208984,112.4455795288086,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,1.107699,1403.675834
1755377088,428136040,1755377088,388855040,62.91643142700195,91.70122528076172,111.9509506225586,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.586906,1404.262740
1755377088,928184338,1755377088,890971520,62.78538131713867,91.77615356445312,111.58921813964844,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.273012,1404.535752
1755377089,428186901,1755377089,439218048,62.7147102355957,91.81710815429688,111.3706283569336,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.236160,1404.771912
1755377089,928213944,1755377089,889331200,62.69528579711914,91.82710266113281,111.33663177490234,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.259353,1405.031266
1755377090,428207105,1755377090,389569024,62.70229721069336,91.82071685791016,111.37431335449219,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.248884,1405.280150
1755377090,928192500,1755377090,891809152,62.72377395629883,91.80702209472656,111.40061950683594,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.226238,1405.506388
1755377091,428162799,1755377091,389927168,62.741905212402344,91.78976440429688,111.37943267822266,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.199824,1405.706212
1755377091,928169083,1755377091,890144512,62.72667694091797,91.7464599609375,111.36900329589844,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.198993,1405.905205
1755377092,428161060,1755377092,390386688,62.545467376708984,91.47853088378906,111.37987518310547,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,0.436408,1406.341613
1755377092,928181354,1755377092,890625024,62.04458236694336,90.75370025634766,111.39105987548828,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,1.290628,1407.632241
1755377093,428159150,1755377093,390859008,61.08335494995117,89.36102294921875,111.39247131347656,1755377085,332905237,62.92041132554209,91.70128343941283,111.36405807162261,2.975299,1410.607539"""


In [ ]:
# Read the provided CSV (embedded as a multiline string), plot pose1/pose2 vs time,
# and show the cumulative distance (distance_sum). Each chart is a separate figure
# (no subplots) and uses default matplotlib styles with pan/zoom toolbar enabled.

import io
import textwrap
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# ---- Load CSV from the user's message ----

# Remove the "truncated" line for clean parsing
csv_text_clean = "\n".join(
    line for line in csv_text.splitlines()
    if "truncated for brevity" not in line and line.strip() != ""
)

df = pd.read_csv(io.StringIO(csv_text_clean))

# Build a monotonic datetime index from sample stamps (sec + nsec)
# (Using UTC for consistency; you can localize later if needed)
sample_ns = df["sample_stamp_sec"] * 1_000_000_000 + df["sample_stamp_nanosec"]
dt_index = pd.to_datetime(sample_ns, unit="ns", utc=True)

# Prepare series for each pose component as 1D numpy arrays to avoid ValueError
p1x = df["pose1_x"].to_numpy().flatten()
p1y = df["pose1_y"].to_numpy().flatten()
p1z = df["pose1_z"].to_numpy().flatten()
p2x = df["pose2_x"].to_numpy().flatten()
p2y = df["pose2_y"].to_numpy().flatten()
p2z = df["pose2_z"].to_numpy().flatten()
dist_sum = df["distance_sum"].to_numpy().flatten()
dt_index = pd.Series(dt_index).to_numpy().flatten()
total_distance = float(dist_sum[-1]) if len(dist_sum) else 0.0

# ---- Plot X over time ----
fig1 = plt.figure(figsize=(9, 5))
ax1 = fig1.gca()
ax1.plot(dt_index, p1x, label="pose1_x")
ax1.plot(dt_index, p2x, label="pose2_x")
ax1.set_title("Pose X vs Time")
ax1.set_xlabel("Time")
ax1.set_ylabel("X [m]")
ax1.legend()
fig1.autofmt_xdate()
plt.show()

# ---- Plot Y over time ----
fig2 = plt.figure(figsize=(9, 5))
ax2 = fig2.gca()
ax2.plot(dt_index, p1y, label="pose1_y")
ax2.plot(dt_index, p2y, label="pose2_y")
ax2.set_title("Pose Y vs Time")
ax2.set_xlabel("Time")
ax2.set_ylabel("Y [m]")
ax2.legend()
fig2.autofmt_xdate()
plt.show()

# ---- Plot Z over time ----
fig3 = plt.figure(figsize=(9, 5))
ax3 = fig3.gca()
ax3.plot(dt_index, p1z, label="pose1_z")
ax3.plot(dt_index, p2z, label="pose2_z")
ax3.set_title("Pose Z vs Time")
ax3.set_xlabel("Time")
ax3.set_ylabel("Z [m]")
ax3.legend()
fig3.autofmt_xdate()
plt.show()

# ---- Plot cumulative distance over time; show total in the title ----
fig4 = plt.figure(figsize=(9, 5))
ax4 = fig4.gca()
ax4.plot(dt_index, dist_sum, label="distance_sum")
ax4.set_title(f"Cumulative Distance vs Time (total = {total_distance:.3f} m)")
ax4.set_xlabel("Time")
ax4.set_ylabel("Distance [m]")
ax4.legend()
fig4.autofmt_xdate()
plt.show()

# Also print the total distance for quick visibility
print(f"Total distance: {total_distance:.3f} m")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    df  # noqa: F821
except NameError:
    file_path = "ros2_ws/pose_distances.csv"  # <--- change to your actual path
    df = pd.read_csv(file_path)

# ---- Build/normalize arrays ----
def col(name):
    if name not in df.columns:
        raise ValueError(f"Missing expected column: {name}")
    return pd.to_numeric(df[name], errors="coerce").to_numpy().flatten()

p1x, p1y, p1z = col("pose1_x"), col("pose1_y"), col("pose1_z")
p2x, p2y, p2z = col("pose2_x"), col("pose2_y"), col("pose2_z")

# Total distance if available
total_distance = None
if "distance_sum" in df.columns:
    ds = pd.to_numeric(df["distance_sum"], errors="coerce").to_numpy().flatten()
    if ds.size:
        total_distance = float(ds[~np.isnan(ds)][-1])

# ---- 3D plot ----
fig = plt.figure(figsize=(9, 7))
ax = fig.add_subplot(111, projection="3d")

ax.plot(p1x, p1y, p1z, label="pose1", linewidth=1.5)
ax.plot(p2x, p2y, p2z, label="pose2", linewidth=1.5)

# Mark start/end
ax.scatter(p1x[0], p1y[0], p1z[0], marker="o", s=40, label="pose1 start")
ax.scatter(p1x[-1], p1y[-1], p1z[-1], marker="^", s=40, label="pose1 end")
ax.scatter(p2x[0], p2y[0], p2z[0], marker="o", s=40, label="pose2 start")
ax.scatter(p2x[-1], p2y[-1], p2z[-1], marker="^", s=40, label="pose2 end")

ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")
ax.set_zlabel("Z [m]")

title = "3D Trajectories: pose1 vs pose2"
if total_distance is not None:
    title += f"  |  total distance = {total_distance:.3f} m"
ax.set_title(title)

ax.legend(loc="best")

# Make axes roughly equal scale
mins = np.array([p1x.min(), p1y.min(), p1z.min(), p2x.min(), p2y.min(), p2z.min()]).reshape(2,3).min(axis=0)
maxs = np.array([p1x.max(), p1y.max(), p1z.max(), p2x.max(), p2y.max(), p2z.max()]).reshape(2,3).max(axis=0)
ranges = maxs - mins
# pad a bit
pad = 0.05 * ranges
mins -= pad
maxs += pad
ax.set_xlim(mins[0], maxs[0])
ax.set_ylim(mins[1], maxs[1])
ax.set_zlim(mins[2], maxs[2])
try:
    ax.set_box_aspect((ranges[0], ranges[1], ranges[2]))
except Exception:
    pass  # older Matplotlib

plt.show()

In [ ]:
# In a Jupyter cell, BEFORE importing pyplot:
%pip install ipympl -q
%matplotlib widget


# ... your existing code that builds df, p1x/p1y/p1z, p2x/p2y/p2z ...

fig = plt.figure(figsize=(9, 7))
ax = fig.add_subplot(111, projection="3d")
ax.plot(p1x, p1y, p1z, label="pose1")
ax.plot(p2x, p2y, p2z, label="pose2")
ax.scatter(p1x[0], p1y[0], p1z[0], s=40, label="pose1 start")
ax.scatter(p1x[-1], p1y[-1], p1z[-1], s=40, label="pose1 end")
ax.scatter(p2x[0], p2y[0], p2z[0], s=40, label="pose2 start")
ax.scatter(p2x[-1], p2y[-1], p2z[-1], s=40, label="pose2 end")
ax.set_xlabel("X [m]"); ax.set_ylabel("Y [m]"); ax.set_zlabel("Z [m]")
ax.legend()
plt.show()  # now it's draggable/zoomable
